### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Set your Azure Subscription](#1)
- [2️⃣ Create the Azure Resource Group](#2)
- [3️⃣ Create an SSH key pair using Azure CLI](#3)
- [4️⃣ Create and connect to an AKS Cluster](#4)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Provide either the tenant and the subscription IDs, before running the lab.
- Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management).
- Ensure the Nodepool VM sizes are compatible with the selected location. If 'aks_node_vm_sizes' is unspecified, the deployment process will identify three available SKUs and attempt to use the first one that successfully deploys.

In [ ]:
import os
import datetime
import json
import subprocess

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"
tenant_id = "<tenant_id>"
subscription_id = "<subscription_id>"
aks_resource_name = "<aks_resource_name>"
ssh_key = f"{deployment_name}-ssh-key"
dns_prefix = f"{deployment_name}-dns"
linux_admin_username = "azureuser"
aks_node_vm_sizes = [ "standard_d2s_v3" ]

<a id='1'></a>
### 1️⃣ Set your Azure Subscription
Use this step to specify which Azure tenant and subscription should be used for subsequent Azure CLI commands.

In [ ]:
! az login --tenant {tenant_id}
! az account set --subscription {subscription_id}

<a id='2'></a>
### 2️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())


<a id='3'></a>
### 3️⃣ Create an SSH key pair using Azure CLI

In [ ]:
import os

# Create an SSH key pair using Azure CLI
! az sshkey create --name {ssh_key} --resource-group {resource_group_name}

# Retrieve the SSH public key content\n",  
ssh_public_key_content = ! az sshkey show --name {ssh_key} --resource-group {resource_group_name} --query publicKey -o tsv  
ssh_public_key = ssh_public_key_content[0] if ssh_public_key_content else ''

os.environ["SSH_PUBLIC_KEY"] = ssh_public_key

<a id='4'></a>
### 4️⃣ Create and connect to an AKS Cluster

In [ ]:
def run_command(command):
    try:
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        return True, output.decode("utf-8")
    except subprocess.CalledProcessError as e:
        return False, e.output.decode("utf-8")
    
# Check if aks_node_vm_sizes is set  
try:  
    aks_node_vm_sizes  
except NameError:  
    aks_node_vm_sizes = None  
  
if not aks_node_vm_sizes:  
    # Get the list of available VM sizes for AKS in the westeurope region  
    success, output = run_command("az vm list-sizes --location {resource_group_location} --output json")  
  
    if success:  
        available_vm_sizes = json.loads(output)  
        # Filter VM sizes with at least 2 CPUs and get the first 3 results  
        filtered_vm_sizes = [vm['name'] for vm in available_vm_sizes if vm['numberOfCores'] >= 2][:3]  
        aks_node_vm_sizes = filtered_vm_sizes  
        print("Filtered VM sizes:", aks_node_vm_sizes)  
    else:  
        print("Failed to get available VM sizes. Error:", output)  
        aks_node_vm_sizes = []  

# Loop through SKUs and attempt to create AKS cluster
for aks_node_vm_size in aks_node_vm_sizes:
    print(f"Trying node vm size: {aks_node_vm_size}")
    
    bicep_parameters = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": {
            "clusterName": { "value": aks_resource_name },
            "location": { "value": resource_group_location },
            "agentVMSize": { "value": aks_node_vm_size },
            "sshRSAPublicKey": { "value":  os.environ.get("SSH_PUBLIC_KEY") },
            "linuxAdminUsername": { "value": linux_admin_username },
            "dnsPrefix": { "value": dns_prefix },
        }
    }
    
    with open('params.json', 'w') as bicep_parameters_file:
        bicep_parameters_file.write(json.dumps(bicep_parameters))

    success, output = run_command(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file \"aks.bicep\" --parameters \"params.json\"")
    if success:
        print("AKS cluster created successfully with node vm size:", aks_node_vm_size)
        break
    else:
        print("Failed to create AKS cluster with node vm size:", aks_node_vm_size)
        print("Error:", output)